<a href="https://colab.research.google.com/github/Guts-war/Alura-imers-o-ia-2025/blob/main/IAtube.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

# Configura o cliente da SDK do Gemini

from google import genai

client = genai.Client()

model = "gemini-2.0-flash"

# Instalar Framework ADK de agentes do Google ################################################
!pip install -q google-adk

from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

##########################################
# --- Agente 1: Buscador de Notícias --- #
##########################################

def agente_buscador(topico, data_de_hoje):
    buscador = Agent(
        name="agente_buscador",
        model="gemini-2.0-flash",
        description="Agente que busca notícias sobre o tópico indicado",
        tools=[google_search],
        instruction="""
        Você é um agente de de pesquisa. A sua tarefa é usar a ferramenta de busca do
        google (google_search) para recuperar as últimas notícias de lançamentos
        muito relevantes sobre o tópico indicado, com base na quantidade e entusiasmo
        das notícias sobre ele. se um tema tiver notícias ou reações entusíasmadas, é
        possível que não seja relevante assim e pode ser substituido por outro que tenha mais.
        Esses lançamentos relevantes devem ser atuais, de no máximo um mês antes da data de hoje
        """
    )
    entrada_do_agente_buscador = f"Tópico: {topico}\nData de hoje: {data_de_hoje}"
    # Executa o agente
    lancamentos_buscados = call_agent(buscador, entrada_do_agente_buscador)
    return lancamentos_buscados

################################################
# --- Agente 2: Planejador de roteiro --- #
################################################
def agente_planejador(topico, lancamentos_buscados):
    planejador = Agent(
        name="agente_planejador",
        model="gemini-2.0-flash",
        # Inserir as instruções do Agente Planejador #################################################
        instruction="""
        Você é um planejador de conteúdo, especialista em rede sociais. Com base
        na lista de lançamentos mais recentes e relevantes buscador, você deve:
        usar a ferramenta de busca do Google (google_search) para criar um plano sobre
        quais são os pontos mais relevantes que poderiamos abordar em um roteiro sobre cada um deles.
        Você também pode usar o (google_search) para demontrar mais informaçoes
        sobre os temas e aprofundar. Ao final, você irá escolher o tema mais relevantes
        entre eles com bases nas suas pesquisas e retornar essse tema, seus pontos, e um
        plano com os assuntos a serem abordados no roteiro que será escrito posteriormente.
        """,
        description="Agente que planeja roteiro",
        tools=[google_search]
    )

    entrada_do_agente_planejador = f"Tópico:{topico}\nLançamentos buscados: {lancamentos_buscados}"
    # Executa o agente
    plano_de_roteiro = call_agent(planejador, entrada_do_agente_planejador)
    return plano_de_roteiro

######################################
# --- Agente 3: Redator do Post --- #
######################################
def agente_redator(topico, plano_de_roteiro):
    redator = Agent(
        name="agente_redator",
        model="gemini-2.0-flash",
        instruction="""
            Você é um Redator Criativo especializado em criar roteiros virais para videos.
            Utilize o tema fornecido no plano de roteiro e os pontos mais relevantes fornecidos e, com base nisso,
            escreva um rascunho de roteiro para descrição do video sobre o tema indicado.
            O roteiro, deve ser engajador, informativo, com linguagem simples.
            """,
        description="Agente redator de roteiro engajadores para Youtube"
    )
    entrada_do_agente_redator = f"Tópico: {topico}\nPlano de roteiro: {plano_de_roteiro}"
    rascunho = call_agent(redator, entrada_do_agente_redator)
    return rascunho

##########################################
# --- Agente 4: Revisor de Qualidade --- #
##########################################
def agente_revisor(topico, rascunho_gerado):
    revisor = Agent(
        name="agente_revisor",
        model="gemini-2.0-flash",
        instruction="""
            Você é um Editor e Revisor de Conteúdo meticuloso, especializado em roteiros para videos, com foco no Youtube.
            Revise o rascunho de roteiro do Youtube abaixo sobre o tópico indicado, verificando clareza, concisão, correção e tom.
            Se o rascunho estiver bom, responda apenas 'O rascunho está ótimo e pronto para publicar!'.
            Caso haja problemas, aponte-os e sugira melhorias.
            """,
        description="Agente revisor de roteiro para videos."
    )
    entrada_do_agente_revisor = f"Tópico: {topico}\nRascunho: {rascunho_roteiro}"
    texto_revisado = call_agent(revisor, entrada_do_agente_revisor)
    return texto_revisado

chat_config = types.GenerateContentConfig(
    system_instruction = """Atue como um 'Assistente de Criação Multimídia'. Auxilie o usuário na elaboração de roteiros e, quando necessário, produza imagens, vídeos e áudios relacionados a esses roteiros, com base nas ideias fornecidas pelo usuário. Propósito e Objetivos: Ajudar os usuários a desenvolver roteiros criativos e originais. Gerar conteúdo multimídia (imagens, vídeos e áudios) que correspondam aos roteiros criados. Facilitar a visualização e a prototipagem de ideias por meio da produção de ativos multimídia. Comportamentos e Regras: 1) Consulta Inicial: a) Apresente-se como um 'Assistente de Criação Multimídia' pronto para ajudar na criação e visualização de ideias. 2) Geração de Conteúdo Multimídia: a) Baseando-se no roteiro, gere imagens que representem cenas ou conceitos chave. b) Crie vídeos curtos que ilustrem sequências de eventos ou demonstrem funcionalidades, se aplicável. c) Produza áudios, como narrações, efeitos sonoros ou músicas de fundo, conforme a necessidade do roteiro. d) Apresente o conteúdo multimídia de forma clara e organizada, facilitando a compreensão pelo usuário. e) Solicite feedback sobre o conteúdo gerado e ofereça revisões ou alternativas, se necessário. Tom Geral: Utilize uma linguagem clara, concisa e inspiradora. Seja colaborativo e receptivo às ideias do usuário. Demonstre entusiasmo pela criação multimídia e pelo auxílio ao usuário em seus projetos."""
)

chat = client.chats.create(model=model, config=chat_config)

from datetime import date

data_de_hoje = date.today().strftime("%d/%m/%Y")

print("🚀 Iniciando o Sistema de Criação de Posts para Instagram com 4 Agentes 🚀")

topico = input("❓ Por favor, digite o TÓPICO sobre o qual você quer criar o roteiro de tendências: ")


if not topico:
    print("❌ Tópico inválido. Por favor, insira um tópico válido.")
else:
      print(f"Maravilha, vamos lá criar o roteiros sobre novidades em {topico}")

      lancamentos_buscados = agente_buscador(topico,  data_de_hoje)
      print("\n--- Resultado do agente 1 (Buscador) ---\n")
      display(to_markdown(lancamentos_buscados))
      print("---------------------------------\n")

      plano_de_roteiro = agente_planejador(topico, lancamentos_buscados)
      print("\n--- Resultado do agente 2 (Planejador) ---\n")
      display(to_markdown(plano_de_roteiro))
      print("---------------------------------\n")

      rascunho_roteiro = agente_redator(topico, plano_de_roteiro)
      print("\n--- Resultado do agente 3 (Redator) ---\n")
      display(to_markdown(rascunho_roteiro))
      print("---------------------------------\n")

      texto_final = agente_revisor(topico, rascunho_roteiro)
      print("\n--- Resultado do agente 4 (Revisor) ---\n")
      display(to_markdown(texto_final))

      prompt = input("Esperando Prompt: ")
while prompt != "/fim":

  resposta = chat.send_message(prompt)

  display(to_markdown(resposta.text))

  print("\n")
  prompt = input("Esperando Prompt: ")